In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from decimal import Decimal
from pyspark.sql.functions import *
import pyspark.sql.functions as F
import pandas as pd
import os
import re
import sys

In [2]:
appName = "DataPrep"
master = "spark://LAPTOP-NKV0VISO.localdomain:7077"
# master = "spark://192.168.56.1:7077"
# master = "local[*]"

In [3]:
# Create Spark session
# spark = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()
spark = SparkSession.builder \
    .config('spark.ui.port','8080') \
    .appName(appName) \
    .master(master) \
    .getOrCreate()

22/04/05 00:00:48 WARN Utils: Your hostname, LAPTOP-NKV0VISO resolves to a loopback address: 127.0.1.1; using 172.31.188.213 instead (on interface eth0)
22/04/05 00:00:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 00:00:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/05 00:00:53 WARN Utils: Service 'SparkUI' could not bind on port 8080. Attempting port 8081.
22/04/05 00:00:53 WARN Utils: Service 'SparkUI' could not bind on port 8081. Attempting port 8082.


In [37]:
dfShopee = spark.read.format("csv")\
                    .option("header", True)\
                    .option("delimiter",";") \
                    .load("hdfs://localhost:9000/data_upload/Shopee.csv")

In [38]:
dfLazada = spark.read.format("csv")\
                    .option("header", True)\
                    .option("delimiter",";") \
                    .load("hdfs://localhost:9000/data_upload/master/Lazada.csv")

In [39]:
dfJD = spark.read.format("csv")\
                    .option("header", True)\
                    .option("delimiter",";") \
                    .load("hdfs://localhost:9000/data_upload/master/JdCentral.csv")

df_shopee = dfShopee.toPandas()
df_shopee = df_shopee.dropna()
for index, row in df_shopee.iterrows():
    row['URL'] =row['URL'].replace(" ", "-")
    row['URL'] =row['URL'].replace("·", "-")
    row['URL'] =row['URL'].replace("\t", "-")

df_Lazada = dfLazada.toPandas()
df_Lazada = df_Lazada.dropna()
for index, row in df_Lazada.iterrows():
    row['Platform'] =row['Platform'].replace(",", "")
    row['Platform'] =row['Platform'].replace('""', "")

dfShopee=spark.createDataFrame(df_shopee)
dfLazada=spark.createDataFrame(df_Lazada)

In [8]:
dfShopee.printSchema()

root
 |-- Name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- historical_sold: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- view_count: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Image: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- rating_star: string (nullable = true)
 |-- shop_location: string (nullable = true)



In [9]:
dfLazada.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Image: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Platform,,,,,,,,,,,,: string (nullable = true)



In [10]:
dfJD.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Subcategory: string (nullable = true)
 |-- Store: string (nullable = true)
 |-- Image: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- enToken: string (nullable = true)
 |-- thToken: string (nullable = true)



In [40]:
dfLazada = dfLazada.withColumnRenamed("Platform,,,,,,,,,,,,","Platform")
dfShopee = dfShopee.withColumnRenamed("Score","Review").withColumnRenamed("rating_star","Score")
dfLazada = dfLazada.where(col("Price").isNotNull())
dfJD = dfJD.withColumn("Score", col("Score") / 20)
dfShopee = dfShopee.where(col("Name").isNotNull())
dfShopee = dfShopee.where(col("Score").isNotNull())

In [41]:
dfShopee = dfShopee.select(col("Name"),col("Price"),col("Review"),col("Score"),col("Image"),col("Category"),col("URL"),col("Platform"),col("brand"),col("historical_sold"),col("view_count"),col("shop_location"))
dfJD = dfJD.select(col("Name"),col("Price"),col("Review"),col("Score"),col("Image"),col("Category"),col("URL"),col("Platform"),col("Store"),col("Subcategory"))
dfLazada = dfLazada.select(col("Name"),col("Price"),col("Review"),col("Score"),col("Image"),col("Category"),col("URL"),col("Platform"))

In [42]:
dfShopee.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Image: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- historical_sold: string (nullable = true)
 |-- view_count: string (nullable = true)
 |-- shop_location: string (nullable = true)



In [43]:
dfLazada.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Image: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Platform: string (nullable = true)



In [44]:
dfJD.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Score: double (nullable = true)
 |-- Image: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Store: string (nullable = true)
 |-- Subcategory: string (nullable = true)



In [45]:
newdf = dfShopee.unionByName(dfJD, allowMissingColumns = True)
newdf = newdf.unionByName(dfLazada, allowMissingColumns = True)
newdf = newdf.dropDuplicates(['URL'])

In [46]:
newdf.show()

+--------------------+-------+------+------------------+--------------------+--------------------+--------------------+--------+--------------------+---------------+----------+--------------------+-----+-----------+
|                Name|  Price|Review|             Score|               Image|            Category|                 URL|Platform|               brand|historical_sold|view_count|       shop_location|Store|Subcategory|
+--------------------+-------+------+------------------+--------------------+--------------------+--------------------+--------+--------------------+---------------+----------+--------------------+-----+-----------+
|4G Pocket WiFi 15...|  829.0|   512|  4.58029197080292|https://cf.shopee...|   อุปกรณ์เน็ตเวิร์ค|https://shopee.co...|  Shopee|No Brand(ไม่มียี่...|            784|      1715|จังหวัดกรุงเทพมหานคร| null|       null|
|9900mAh ถ่านชาร์จ...|   10.0|   410| 4.712628865979381|https://cf.shopee...|  อุปกรณ์เสริมมือถือ|https://shopee.co...|  Shopee|No Brand

In [20]:
# newdf = newdf.withColumnRenamed("Category","SubCategory")

In [23]:
# newdf = newdf.withColumn("Category", lit(''))

In [30]:
newdf

DataFrame[Name: string, Price: string, Review: string, Score: string, Image: string, Category: string, URL: string, Platform: string, brand: string, historical_sold: string, view_count: string, shop_location: string, Store: string, Subcategory: string]

In [31]:
count_df = newdf.select('Category')\
    .groupBy('Category')\
    .count()\
    .sort('count', ascending=False)

df_new = count_df.toPandas()
df_new.to_csv('test.csv',encoding='utf-8-sig',index= False)

In [33]:
count_df.show(100000)


+--------------------+-----+
|            Category|count|
+--------------------+-----+
|คอมพิวเตอร์และสำน...|25221|
|อุปกรณ์อิเล็กทรอน...|21054|
|โทรศัพท์มือถือและ...| 9149|
|      โทรศัพท์มือถือ| 1147|
|            แท็บเล็ต| 1097|
|         กล้องคอมแพค| 1076|
|คอมพิวเตอร์แบบตั้...| 1040|
|    กล้องมิลเลอร์เลส| 1038|
|         กล้องวิดีโอ| 1035|
|          กล้อง DSLR| 1035|
|            แล็ปท็อป| 1034|
|      กล้องอินสแตนท์| 1033|
|กล้องและระบบรักษา...| 1029|
|                โดรน| 1028|
|เครื่องเล่นเกมคอมโซล| 1027|
|   อุปกรณ์เน็ตเวิร์ค|   40|
|อุปกรณ์จัดเก็บข้อมูล|   20|
|        กล้องแอคชั่น|   20|
|     เคสและซองมือถือ|   20|
|กล้องและอุปกรณ์ถ่...|   20|
|อุปกรณ์สำหรับเล่นเกม|   20|
|           ซอฟต์แวร์|   20|
|อุปกรณ์เสริมคอมพิ...|   20|
| อุปกรณ์กันรอยหน้าจอ|   20|
|               เลนส์|   20|
|แล็ปท็อปและคอมตั้...|   20|
|               ฟิล์ม|   20|
|      แบตเตอรี่สำรอง|   20|
|       เมมโมรี่การ์ด|   20|
|   อุปกรณ์เสริมกล้อง|   20|
|มือถือและอุปกรณ์เ...|   20|
|  อุปกรณ์เสริ

In [50]:
count_df = count_df.toPandas()
# count_df.to_csv('CT2.csv',encoding='utf-8-sig',index= False)

In [47]:
keyword = []
thkeyword = []
numkeyword = []
string = ""
thstring = ""
numstring = ""
p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'
for row in newdf.rdd.collect():
    line = re.sub(r"[^A-Za-z-?\d+\.?\d*]", " ", row["Name"].strip())
    thline = re.sub(r"[^\u0E00-\u0E7F-?\d+\.?\d*]", " ", row["Name"].strip())
    words = line.split()
    thwords = thline.split()
    for word in words:
        string += word + " "
        if re.search(p, word) is not None:
            for catch in re.finditer(p, word):
                numstring += catch[0] + " " # catch is a match object
    for thword in thwords: 
        thstring += thword + " "
    keyword.append(string)
    thkeyword.append(thstring)
    numkeyword.append(numstring)
    string = ""
    thstring = ""
    numstring = ""

In [48]:
df_new = newdf.toPandas()

In [49]:
df_new["enToken"] = keyword
df_new["thToken"] = thkeyword

In [50]:
df_new['Score'] = df_new['Score'].fillna(0)
df_new['URL'] = df_new['URL'].dropna()
df_new['Platform'] = df_new['Platform'].dropna()
df_new['enToken'] = df_new['enToken'].replace('' ,'NULL')
df_new['thToken'] = df_new['thToken'].replace('' ,'NULL')

In [51]:
for index, row in df_new.iterrows():
    row['Price'] = row['Price'].translate({ ord(c): None for c in '"฿,'  })


In [53]:
len(df_new)

68527

In [54]:
df_new.to_csv('JD+Shopee+Lazada.csv',encoding='utf-8-sig',sep=';',index= False)